This is a data cleanup/merge notebook. 

In [1]:
#Import libraries
import pandas as pd
import numpy as np

In [3]:
#Load Data Frames
Unemployment_data = pd.read_csv("unemployment.csv")
financial_data = pd.read_csv("Financial Data.csv")
consumption_data = pd.read_csv("coffee_consumption.csv", header=3)
gdp_perhw_data = pd.read_csv("gdp_hrwkd.csv")
srini_data = pd.read_csv("Coffe_Consuption_clean.csv")
consumption_data.head

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa9 in position 0: invalid start byte

In [3]:
#Remove N/A rows from Consumption Data Frame

consumption_clean_rows = consumption_data.dropna(thresh=11)

#Filter for data for 1999-2013
consumption_clean = consumption_clean_rows.dropna(axis=1)
consumption_clean.head()


,Calendar years,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
1,European Union,"39,283","38,351","38,591","39,038","40,212","41,743","39,877","41,586","41,141","40,700","40,042","41,196","40,756","40,979","41,585"
2,Austria,"1,123",875,"1,013",926,720,996,772,612,847,908,886,903,"1,117","1,269","1,249"
3,Belgium,834,"1,133",884,"1,484","1,579","1,396","1,158","1,537","1,103",650,934,871,934,915,"1,245"
5,Bulgaria,363,275,350,341,397,364,430,420,364,445,409,395,360,376,419
6,Croatia,337,327,337,355,375,370,374,386,386,377,366,370,367,360,387


In [4]:
#Rename Calendar years as Country Name
consumption_clean.rename(index=str, columns={"Calendar years": "Country Name"}, inplace=True)
consumption_clean.columns


C:\Users\mcoats\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Index(['Country Name', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013'],
      dtype='object')

In [5]:
#Move Years to a single column
consumption_clean2 = pd.melt(consumption_clean, id_vars=['Country Name'], var_name="Year", value_name="Coffee Consumed")
consumption_clean2



,Country Name,Year,Coffee Consumed
0,European Union,1999,"39,283"
1,Austria,1999,"1,123"
2,Belgium,1999,834
3,Bulgaria,1999,363
4,Croatia,1999,337
5,Cyprus,1999,54
6,Czech Republic,1999,558
7,Denmark,1999,869
8,Estonia,1999,95
9,Finland,1999,979


In [6]:
#Remove Extra spaces in front of Country Names
consumption_clean2["Country Name"] = consumption_clean2["Country Name"].str.strip()




In [7]:
#View Financial Data Set
financial_data.head()


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [8]:
#Drop Unneeded Columns
financial_data_clean = financial_data.drop(columns=['sex', 'age', 'suicides_no',
                                                    'suicides/100k pop', 'country-year', 'HDI for year', 'generation'])
financial_data_clean.columns


Index(['country', 'year', 'population', ' gdp_for_year ($) ',
       'gdp_per_capita ($)'],
      dtype='object')

In [9]:
#Group by Country and Year
financial_group = financial_data_clean.groupby(["country", "year", " gdp_for_year ($) ",
                                                "gdp_per_capita ($)"])

total_pop = financial_group.sum()
total_pop.reset_index(inplace=True)
total_pop.head()

,country,year,gdp_for_year ($),gdp_per_capita ($),population
0,Albania,1987,"2,156,624,900",796,2709600
1,Albania,1988,"2,126,000,000",769,2764300
2,Albania,1989,"2,335,124,988",833,2803100
3,Albania,1992,"709,452,584",251,2822500
4,Albania,1993,"1,228,071,038",437,2807300


In [10]:
#rename country to Country Name and year to Year
total_pop.rename(index=str, columns={"country": "Country Name","year": "Year" }, inplace=True)


In [11]:
total_pop.head()

,Country Name,Year,gdp_for_year ($),gdp_per_capita ($),population
0,Albania,1987,"2,156,624,900",796,2709600
1,Albania,1988,"2,126,000,000",769,2764300
2,Albania,1989,"2,335,124,988",833,2803100
3,Albania,1992,"709,452,584",251,2822500
4,Albania,1993,"1,228,071,038",437,2807300


In [12]:
#View GDP per Hour Worked file
gdp_perhw_data.dtypes


LOCATION       object
Country        object
INDICATOR      object
SUBJECT        object
MEASURE        object
FREQUENCY      object
TIME            int64
Value         float64
Flag Codes     object
dtype: object

In [13]:
#Drop Unneeded Columns
gdph_clean = gdp_perhw_data.drop(columns=['LOCATION', 'INDICATOR', 'SUBJECT',
                                                    'FREQUENCY', 'Flag Codes'])
#rename Country to Country Name
gdph_clean.rename(index=str, columns={"Country": "Country Name", "TIME": "Year"}, inplace=True)
gdph_clean.head()

,Country Name,MEASURE,Year,Value
0,Australia,USD,1970,26.793218
1,Australia,USD,1971,27.462937
2,Australia,USD,1972,28.067364
3,Australia,USD,1973,28.645413
4,Australia,USD,1974,28.189555


In [14]:
#Remove extra spaces in front of country name
gdph_clean["Country Name"] = gdph_clean["Country Name"].str.strip()

#Convert Year to object
gdph_clean["Year"] = gdph_clean["Year"].astype(object)



In [15]:
#View Unemployment data
Unemployment_data.head()

,Country Name,Country Code,Indicator Name,Year,Value
0,Arab World,ARB,"Unemployment, total (% of total labor force)",1960,NaN
1,Caribbean small states,CSS,"Unemployment, total (% of total labor force)",1960,NaN
2,Central Europe and the Baltics,CEB,"Unemployment, total (% of total labor force)",1960,NaN
3,Early-demographic dividend,EAR,"Unemployment, total (% of total labor force)",1960,NaN
4,East Asia & Pacific,EAS,"Unemployment, total (% of total labor force)",1960,NaN


In [16]:
#Drop Uneeded Columns
Unemployment_data.drop(columns=['Country Code', 'Indicator Name'], inplace = True)
Unemployment_data.head()

,Country Name,Year,Value
0,Arab World,1960,NaN
1,Caribbean small states,1960,NaN
2,Central Europe and the Baltics,1960,NaN
3,Early-demographic dividend,1960,NaN
4,East Asia & Pacific,1960,NaN


In [17]:
#Rename Value to %Unemployed
Unemployment_data.rename(index=str, columns={"Value": "% Unemployed"}, inplace=True)
Unemployment_data.head()

,Country Name,Year,% Unemployed
0,Arab World,1960,NaN
1,Caribbean small states,1960,NaN
2,Central Europe and the Baltics,1960,NaN
3,Early-demographic dividend,1960,NaN
4,East Asia & Pacific,1960,NaN


In [18]:
consumption_clean2['Country Name'] = consumption_clean2['Country Name'].astype(str)
consumption_clean2['Year'] = consumption_clean2['Year'].astype(str)
consumption_clean2.dtypes

Country Name       object
Year               object
Coffee Consumed    object
dtype: object

In [19]:
gdph_clean['Country Name'] = gdph_clean['Country Name'].astype(str)
gdph_clean['Year'] = gdph_clean['Year'].astype(str)
gdph_clean.dtypes

Country Name     object
MEASURE          object
Year             object
Value           float64
dtype: object

In [20]:
total_pop['Year'] = total_pop['Year'].astype(str)
total_pop.dtypes

Country Name          object
Year                  object
 gdp_for_year ($)     object
gdp_per_capita ($)     int64
population             int64
dtype: object

In [21]:
total_pop['Year'] = total_pop['Year'].astype(str)
Unemployment_data.dtypes

Country Name     object
Year             object
% Unemployed    float64
dtype: object

In [22]:
#Merge Data Sets 
combine_1 = pd.merge(consumption_clean2, gdph_clean, how='left', on=['Country Name', 'Year'])
combine_1.head()


,Country Name,Year,Coffee Consumed,MEASURE,Value
0,European Union,1999,"39,283",NaN,NaN
1,Austria,1999,"1,123",USD,43.673565
2,Austria,1999,"1,123",IDX2010,84.786935
3,Belgium,1999,834,USD,56.315921
4,Belgium,1999,834,IDX2010,89.165425


In [23]:
combine_2 = pd.merge(combine_1, total_pop, how='left', on=['Country Name', 'Year'])
combine_2.head()

,Country Name,Year,Coffee Consumed,MEASURE,Value,gdp_for_year ($),gdp_per_capita ($),population
0,European Union,1999,"39,283",NaN,NaN,NaN,NaN,NaN
1,Austria,1999,"1,123",USD,43.673565,"217,185,787,343",28722.0,7561609.0
2,Austria,1999,"1,123",IDX2010,84.786935,"217,185,787,343",28722.0,7561609.0
3,Belgium,1999,834,USD,56.315921,"260,202,429,150",26969.0,9648277.0
4,Belgium,1999,834,IDX2010,89.165425,"260,202,429,150",26969.0,9648277.0


In [24]:
combine_3 = pd.merge(combine_2, Unemployment_data, how='left', on=['Country Name', 'Year'])
combine_3.head()

,Country Name,Year,Coffee Consumed,MEASURE,Value,gdp_for_year ($),gdp_per_capita ($),population,% Unemployed
0,European Union,1999,"39,283",NaN,NaN,NaN,NaN,NaN,9.569237
1,Austria,1999,"1,123",USD,43.673565,"217,185,787,343",28722.0,7561609.0,3.700000
2,Austria,1999,"1,123",IDX2010,84.786935,"217,185,787,343",28722.0,7561609.0,3.700000
3,Belgium,1999,834,USD,56.315921,"260,202,429,150",26969.0,9648277.0,8.600000
4,Belgium,1999,834,IDX2010,89.165425,"260,202,429,150",26969.0,9648277.0,8.600000


In [25]:
combine_clean = combine_3.loc[combine_3["MEASURE"] == "USD", :]
combine_clean.head()

,Country Name,Year,Coffee Consumed,MEASURE,Value,gdp_for_year ($),gdp_per_capita ($),population,% Unemployed
1,Austria,1999,"1,123",USD,43.673565,"217,185,787,343",28722.0,7561609.0,3.7
3,Belgium,1999,834,USD,56.315921,"260,202,429,150",26969.0,9648277.0,8.6
6,Bulgaria,1999,363,USD,12.163121,"13,495,075,366",1715.0,7866600.0,14.1
8,Croatia,1999,337,USD,21.247108,"23,386,945,597",5459.0,4283800.0,13.5
10,Czech Republic,1999,558,USD,21.945953,"64,867,483,193",6606.0,9819215.0,8.7


In [26]:
combine_clean.count()

Country Name          434
Year                  434
Coffee Consumed       434
MEASURE               434
Value                 434
 gdp_for_year ($)     430
gdp_per_capita ($)    430
population            430
% Unemployed          419
dtype: int64

In [27]:
#Drop NA's
combine_clean2 = combine_clean.dropna(how='any')
combine_clean2.count()

Country Name          416
Year                  416
Coffee Consumed       416
MEASURE               416
Value                 416
 gdp_for_year ($)     416
gdp_per_capita ($)    416
population            416
% Unemployed          416
dtype: int64

In [28]:
#Write to csv
combine_clean2.to_csv("project data.csv", index=False)

In [29]:
srini_data.head()
srini_data.rename(index=str, columns={"Value": "Coffee Consumed"}, inplace=True)

srini_data.head()

,Country Name,Year,Coffee Consumed
0,Austria,1990,"1,340"
1,Bulgaria,1990,102
2,Cyprus,1990,31
3,Czech Republic,1990,659
4,Denmark,1990,865


In [30]:
srini_data['Year'] = srini_data['Year'].astype(str)
srini_data['Country Name'] = srini_data['Country Name'].astype(str)
srini_data.dtypes

Country Name       object
Year               object
Coffee Consumed    object
dtype: object

In [31]:
scombine_1 = pd.merge(srini_data, gdph_clean, how='left', on=['Country Name', 'Year'])
scombine_1.head()

,Country Name,Year,Coffee Consumed,MEASURE,Value
0,Austria,1990,"1,340",NaN,NaN
1,Bulgaria,1990,102,NaN,NaN
2,Cyprus,1990,31,NaN,NaN
3,Czech Republic,1990,659,NaN,NaN
4,Denmark,1990,865,USD,44.77973


In [32]:
scombine_2 = pd.merge(scombine_1, total_pop, how='left', on=['Country Name', 'Year'])
scombine_2.head()

,Country Name,Year,Coffee Consumed,MEASURE,Value,gdp_for_year ($),gdp_per_capita ($),population
0,Austria,1990,"1,340",NaN,NaN,"166,463,386,663",23020.0,7231353.0
1,Bulgaria,1990,102,NaN,NaN,"20,632,090,909",2451.0,8418500.0
2,Cyprus,1990,31,NaN,NaN,NaN,NaN,NaN
3,Czech Republic,1990,659,NaN,NaN,"40,477,403,220",4168.0,9710900.0
4,Denmark,1990,865,USD,44.77973,NaN,NaN,NaN


In [33]:
scombine_3 = pd.merge(scombine_2, Unemployment_data, how='left', on=['Country Name', 'Year'])
scombine_3.head()

,Country Name,Year,Coffee Consumed,MEASURE,Value,gdp_for_year ($),gdp_per_capita ($),population,% Unemployed
0,Austria,1990,"1,340",NaN,NaN,"166,463,386,663",23020.0,7231353.0,NaN
1,Bulgaria,1990,102,NaN,NaN,"20,632,090,909",2451.0,8418500.0,NaN
2,Cyprus,1990,31,NaN,NaN,NaN,NaN,NaN,NaN
3,Czech Republic,1990,659,NaN,NaN,"40,477,403,220",4168.0,9710900.0,NaN
4,Denmark,1990,865,USD,44.77973,NaN,NaN,NaN,NaN


In [34]:
scombine_clean = scombine_3.loc[scombine_3["MEASURE"] == "USD", :]

In [35]:
#Drop NA's
scombine_clean2 = scombine_clean.dropna(how='any')
scombine_clean2.count()

Country Name          432
Year                  432
Coffee Consumed       432
MEASURE               432
Value                 432
 gdp_for_year ($)     432
gdp_per_capita ($)    432
population            432
% Unemployed          432
dtype: int64

In [36]:
scombine_clean2.to_csv("project data(srini data).csv", index=False)